In [1]:
#librerias
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import json
import pandas as pd
from datetime import datetime
import numpy as np
import re
from selenium.common.exceptions import TimeoutException

In [2]:
import FunClear  # Funciones creadas

Se importo correctamente la funcion EliminarInusuales y Agrupar


In [3]:
##Nuevas
def buscar(elemento,busqueda):
    WebDriverWait(driver, 10)\
        .until(EC.element_to_be_clickable((By.CSS_SELECTOR,elemento)))\
        .send_keys(busqueda)
    
def Click(elemento):
    WebDriverWait(driver, 5)\
        .until(EC.element_to_be_clickable((By.CSS_SELECTOR,elemento))).click()
  

def LeerJson(path):
    
    WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.XPATH,path)))
    Json_Jumbo=driver.find_element("xpath",path).get_attribute("text")
    Json_Jumbo = json.loads(Json_Jumbo)
    DatosProducto = Json_Jumbo['itemListElement'] # Se toma el Json anidado que contiene los datos del producto
    return DatosProducto



In [4]:
  
#Funcion para convertir palabras con acentos en palabras sin acentos
def categoria(producto):
    
    m =producto.find(" ")
    remplazo = producto[:m].maketrans("áéíóú", "aeiou","*123456789") #Remplazo palabras que contengan acentos y borrar caracteres *123456789
    producto = producto[:m].translate(remplazo).upper() #Uso translate para poder observar correctamente
    return producto



In [5]:
#Funcion para obtener la cantidad de cada producto
def cantidadx(producto):
    
    producto = re.findall(r'( x.+\d{0,6}.\w{1,3}|\d{1,5}..\w{0,5}| \d{1,3}\D\d{1,3}.\w{1,3})',producto) #Expresion regular para obtener las cantidades de los productos
    producto = " ".join(producto).replace('x', '') #Eliminamos la x en la cantidad de productos y pasamos los datos a str

    #----Remplazar valores inusuales----
    producto = re.sub(' \.', '.',producto) #Eliminamos espacios vacios en valores decimales
    
    return producto #Retornar en minusculas y sin espacios al inicio y al final 


def EliminarValoresSeparador(producto):
    
    #Primero se remplaza valores de str que son inusuales
    producto = producto.replace(';', '')
    producto = producto.replace('\r\n', '')
    producto = producto.replace('\n', '')
    producto = producto.replace('&quot', '')
    
    return producto

#Función para separar el numero de la unidad de medida en el nombre
def separador(producto):

    patron =r'(\d{1,5})([a-z]{0,7})' #Expresion regular para dar espacio entre numeros y letras
    producto = re.sub(patron,r'\1 \2',producto) #Separamos los numeros que estan junto a palabras de unidad de medida
    producto = re.sub(' +', ' ',producto) #Esta linea solo deja un espacio entre palabras eliminando espacios inecesarios
    producto = re.sub(' \,', '.',producto) #Eliminamos espacios vacios en valores decimales
    producto = re.sub(' \.', '.',producto) #Eliminamos espacios vacios en valores decimales
    producto = producto.replace('-', '')
    
    return producto.lower()


In [6]:
#Ruta de chorme compatible 
driver_path = "C:\\Users\\danie\\OneDrive\\Documentos\\Notebooks\\chromedriver.exe"
## Opciones con las que inicia google chrome 
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument("--headless")
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

s = Service(driver_path)#Se agrega el path como objeto para evitar errores 


# Main

## Función buscar:
- Recibe como parametro el input class del cuadro de busqueda de la pagina y el producto a buscar
- Se antepone input antes de colocar el input class
- Si el input class tiene espacios estos se remplazan por puntos

## Funcion Click
- Recibe como parametro el elemento clikeable anteponiendo el formato(div, footer, etc)


In [19]:
###### PRUEBA RESISTENCIA CODIGO ######
from selenium.webdriver.support.select import Select

driver = webdriver.Chrome(service=s)
driver.maximize_window()
driver.implicitly_wait (10)
driver.get("https://www.tiendasjumbo.co/") 


#--------------------------------Bloque de elección de ciudad y almacen----------------------------------------#
#Click en configurar metodo de entrega
driver.find_element(By.CLASS_NAME, 'mv0.tiendasjumboqaio-delivery-modal-3-x-textDeliveryTrigger.tiendasjumboqaio-delivery-modal-3-x-textSecondDeliveryTrigger').click()

#Ingresar email
input_class="input.vtex-styleguide-9-x-input.ma0.border-box.vtex-styleguide-9-x-hideDecorators.vtex-styleguide-9-x-noAppearance.br2.w-100.bn.outline-0.bg-base.c-on-base.b--muted-4.hover-b--muted-3.t-body.ph5"
buscar(input_class,"daniel25a@gmail.com")

#Click en enviar
driver.find_element(By.CLASS_NAME,"vtex-button__label.flex.items-center.justify-center.h-100.ph6.w-100.border-box").click()

#Click en recoger en tienda
driver.find_element(By.ID, 'Recoge_en_tienda').click()

#Desplegar la lista de departamentos
driver.find_element(By.CLASS_NAME, 'vtex-styleguide-9-x-container.vtex-dropdown__container.br2.relative.bw1.bg-base.ba.h-regular.hover-b--muted-3.b--muted-4.t-body').click()

# #Elegir departamento boyaca
path = "/html/body/div[12]/div[1]/div/div[2]/div/div/div/div[5]/div[1]/div[1]/div/label/div/select"
Select(driver.find_element("xpath",path)).select_by_visible_text("Boyacá")

# #Desplegar la lista de ciudades
driver.find_element(By.CLASS_NAME, 'vtex-styleguide-9-x-container.vtex-dropdown__container.br2.relative.bw1.bg-base.ba.h-regular.hover-b--muted-3.b--muted-4.t-body').click()
path2 = "/html/body/div[12]/div[1]/div/div[2]/div/div/div/div[5]/div[1]/div[2]/div/label/div/select"
Select(driver.find_element("xpath",path2)).select_by_visible_text("Tunja")
time.sleep(3)

# #Desplegar la lista de tiendas
driver.find_element("xpath", '/html/body/div[12]/div[1]/div/div[2]/div/div/div/div[5]/div[2]/div/div/label/div').click()
path3 = "/html/body/div[12]/div[1]/div/div[2]/div/div/div/div[5]/div[2]/div/div/label/div/select"
Select(driver.find_element("xpath",path3)).select_by_visible_text("Unicentro")
time.sleep(1)

#Click en confirmar 
driver.find_element("xpath", '/html/body/div[12]/div[1]/div/div[2]/div/div/div/div[5]/div[3]/div[2]/button/div').click()
time.sleep(12)

#-----------------------------------------Fin de elección de ciudad y almacen----------------------------------------------#

#-#-#-#-#-#-##-#-#-#-#-#-##-#-#-#-#-#-##--Inicio de busqueda de productos--#-#-#-#-#-##-#-#-#-#-#-##-#-#-#-#-#-##-#-#-#-#-#-#
ListaProducto=  ['Arroz','Leche','Harina','Pasta','Azucar','aseo','frutas','verduras','Huevos','Cafe','Sal','frijol y lenteja','Carne']
CantMas = dict([('Arroz', 3),('Leche', 2),('Harina', 2),('Pasta', 2),('Azucar', 2),('aseo', 1),('frutas', 1),('verduras', 1),('Huevos', 1),('Cafe', 1),('Sal', 1),('frijol y lenteja', 2),('Carne', 2)])

Json_Total=[]
time.sleep(2)
numero_intentos = 2
try:
    for i in range(len(ListaProducto)):
        time.sleep(2)
    #Buscar producto
        producto = ListaProducto[i]
        input_class = 'input.vtex-styleguide-9-x-input.ma0.border-box.vtex-styleguide-9-x-hideDecorators.vtex-styleguide-9-x-noAppearance.br2.br-0.br--left.w-100.bn.outline-0.bg-base.c-on-base.b--muted-4.hover-b--muted-3.t-body.pl5'
        try:
            buscar(input_class,producto)
            time.sleep(2)
        except:
            print('Entro en bucle de buscar producto'+producto)
            driver.get("https://www.tiendasjumbo.co/") 
            buscar(input_class,producto)

    #Click en buscar
        Element_clickeable = 'footer.vtex-search-2-x-tileListFooter' 
        try:
            Click(Element_clickeable)
            time.sleep(1)
        except:
            try:
                print('Entro en except de click '+producto)
                time.sleep(1)
                driver.get("https://www.tiendasjumbo.co/")
                time.sleep(1) 
                buscar(input_class,producto)
                time.sleep(2)
                Click(Element_clickeable)
            except:
                i = i+1
                continue
    #Scroll
        Element_clickeable2 = 'a.vtex-button.bw1.ba.fw5.v-mid.relative.pa0.lh-solid.br2.min-h-small.t-action--small.bg-action-primary.b--action-primary.c-on-action-primary.hover-bg-action-primary.hover-b--action-primary.hover-c-on-action-primary.pointer.inline-flex.items-center.no-underline'
        path= '/html/body/div[2]/div/div[1]/div/div[4]/div/div/section/div[2]/div/div[4]/div/div[2]/div/div/div[2]/div/div[5]/div/div/div/script'      

        try:
            for j in range(CantMas[ListaProducto[i]]):
                Click(Element_clickeable2)
                time.sleep(2)
            time.sleep(1)
            Json_Total += LeerJson(path) #Esto deja un JSON con el que se trabajara los datos

        except:
            try: 
                path2= '/html/body/div[3]/div/div[1]/div/script'      
                print('Entro en try de scroll 1 '+producto)
                Json_Total += LeerJson(path2)  #Guardar en Json 
                driver.get("https://www.tiendasjumbo.co/")
            except:
                print('Entro en try de scroll 2 '+producto)
                driver.get("https://www.tiendasjumbo.co/")                            
        #limpiar
        if(i>=1):  
            driver.get("https://www.tiendasjumbo.co/") 
            
except TimeoutException:
    driver.set_page_load_timeout(10)
    pass
    print('El proceso finalizo con algo inusual')
    
finally:
    print('Finalizo correctamente el proceso')
    driver.close()

Entro en try de scroll 1 Azucar
Entro en try de scroll 1 Huevos
Finalizo correctamente el proceso


# Procesamiento de datos

In [ ]:
#Se obtienen los datos del segundo nivel de profundidad del json y se pasan a un dataframe
Producto_1 = pd.json_normalize(Json_Total) 
#Se accede al 3 nivel de produndidad del json 
JsonAnidado=[] #Se crea una lista vacia para llenarla con el for

for i in Producto_1['item.offers.offers']:
    JsonAnidado +=i 
    
#Se normaliza el nivel 3 de profundidad del json    
Producto_2= pd.json_normalize(JsonAnidado) # Hacemos join de los 2 dataframe creados
Productos_T = pd.concat([Producto_1, Producto_2], axis = 1) 

In [ ]:
#Seleccionamos las columnas que nos interesan
Productos_T = Productos_T[['sku','item.name','item.brand.name','item.description','priceCurrency','price','priceValidUntil','item.@id','item.image','seller.name']]
#Renombramos las columnas 
Productos_T.columns = ['Codigo_Pro','Nombre', 'Marca', 'Descripcion', 'Moneda', 'Precio','Precio_Valido_Hasta','Url_producto', 'Url_Imagen','Almacen']

In [ ]:
#Obtenemos la fecha actual
now = datetime.now()
format = now.strftime('%Y-%m-%d')
NombreAlmacen='Jumbo-'+format
Ruta = "C:\\Users\\danie\\OneDrive\\Documentos\\Proyecto_Supermercados_GIT\\Proyecto_Supermercados\\ProductData\\"

# Limpieza de datos

In [ ]:
#Eliminar valores como (;,/n,/r/n) para evitar problemas de formato al momento de exportar archivo csv
Productos_T['Nombre']=Productos_T['Nombre'].apply(lambda x: EliminarValoresSeparador(x))
Productos_T['Descripcion']=Productos_T['Descripcion'].apply(lambda x: EliminarValoresSeparador(x))
Productos_T['Marca']=Productos_T['Marca'].apply(lambda x: EliminarValoresSeparador(x))
Productos_T['Nombre']=Productos_T['Nombre'].apply(lambda x: separador(x))
Productos_T['Cantidad']=Productos_T['Nombre'].apply(lambda x: cantidadx(x))
Productos_T['Fecha']= now.strftime('%Y-%m-%d')
Productos_T['Categoria']=Productos_T['Nombre'].apply(lambda x: categoria(x))
Productos_T['Cantidad']=Productos_T['Cantidad'].apply(lambda x: FunClear.EliminarInusuales(x))
Productos_T['Categoria']=Productos_T['Categoria'].apply(lambda x: FunClear.Agrupar(x))
Productos_T['Ciudad']='Tunja'

## Exporta archivo json


In [ ]:
#Exportar archivo CSV
Productos_T.to_csv(f'{Ruta}{NombreAlmacen}.csv', index=False,sep='|',encoding='utf-8')

In [ ]:

# Ver todo el contenido de un dataframe
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.width = None
pd.options.display.max_colwidth = None

In [ ]:
#Productos_T